# Modeling, Predictions and Analysis of CO2 Emissions Data

### Libraries

In [1]:
# load the required libraries

import pandas as pd
import numpy as np
from sklearn import linear_model, datasets
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from scipy.stats import spearmanr, pearsonr
import matplotlib.pyplot as plt
%matplotlib inline
import plotly
import plotly.plotly as py
py.sign_in('','')
import warnings
warnings.filterwarnings("ignore")

### Data Load

In [2]:
# function to load the data

def read_data(filename):
    
    df = pd.read_csv(filename,index_col=[0,1], header=2)
    cols = []

    for i in range(1960,2015):
        cols.append(str(i))

    df = df[cols[:len(cols)]]
    df = df.reset_index()

    country_metadata = pd.read_csv("Metadata_Country.csv", header=0, usecols=['Country Code','Region','IncomeGroup'])
    country_metadata = country_metadata.dropna()

    # merge the dataframe with country metadata (Data Source: WorldBank {https://data.worldbank.org})
    # to get only the countries and remove subgroups like Europe, Middle East etc

    df_merged = df.merge(country_metadata,on=['Country Code'])
    df_merged = df_merged.fillna(0)
    
    return df_merged

In [3]:
# load the data (Data Source: WorldBank {https://data.worldbank.org})

co2_emissions_total = read_data("CO2Emissions_KT.csv")
gdp_current_usd     = read_data("GDP_CurrentUSD.csv")
population          = read_data("WorldPopulation_Total.csv")

### Data Preprocessing

In [4]:
# remove unnecessary columns to get country and year-wise gdp and population data

gdp_current_usd = gdp_current_usd.drop(['Country Name','Region','IncomeGroup'], axis=1)
population = population.drop(['Country Name','Region','IncomeGroup'], axis=1)

In [5]:
# check first few rows of the data

gdp_current_usd.head(1)

,Country Code,1960,1961,1962,1963,1964,1965,1966,1967,1968,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014
0,ABW,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.331006e+09,2.421475e+09,2.623726e+09,2.791961e+09,2.498933e+09,2.467704e+09,2.584464e+09,0.0,0.0,0.0


### Data Transformation

In [6]:
# melt the dataframes so that the data can be subsequently merged

co2_emissions_total_melt = pd.melt(co2_emissions_total,id_vars=['Country Name','Country Code','Region','IncomeGroup'],
                                   var_name='year',value_name='co2_emissions')

gdp_current_usd_melt = pd.melt(gdp_current_usd,id_vars=['Country Code'],
                               var_name='year',value_name='gdp_current_usd')

population_melt = pd.melt(population,id_vars=['Country Code'],
                          var_name='year',value_name='population')

In [7]:
# check the melted dataframe

gdp_current_usd_melt.head(1)

,Country Code,year,gdp_current_usd
0,ABW,1960,0.0


### Merging Dataframes

In [8]:
# merge gdp and population data with co2 emissions data on Country Code and year columns

co2_gdp_population_data =  co2_emissions_total_melt.merge(gdp_current_usd_melt, 
                                on=['Country Code','year']).merge(population_melt,
                                on=['Country Code','year'])

In [9]:
# check the merged dataframe

co2_gdp_population_data.head()

,Country Name,Country Code,Region,IncomeGroup,year,co2_emissions,gdp_current_usd,population
0,Aruba,ABW,Latin America & Caribbean,High income,1960,0.000,0.000000e+00,54211.0
1,Afghanistan,AFG,South Asia,Low income,1960,414.371,5.377778e+08,8996351.0
2,Angola,AGO,Sub-Saharan Africa,Lower middle income,1960,550.050,0.000000e+00,5643182.0
3,Albania,ALB,Europe & Central Asia,Upper middle income,1960,2024.184,0.000000e+00,1608800.0
4,Andorra,AND,Europe & Central Asia,High income,1960,0.000,0.000000e+00,13411.0


### Data Export

In [10]:
# remove the rows with zero values

co2_gdp_population_data = co2_gdp_population_data[(co2_gdp_population_data != 0).all(1)]

# check the information about the dataframe

co2_gdp_population_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8354 entries, 1 to 11934
Data columns (total 8 columns):
Country Name       8354 non-null object
Country Code       8354 non-null object
Region             8354 non-null object
IncomeGroup        8354 non-null object
year               8354 non-null object
co2_emissions      8354 non-null float64
gdp_current_usd    8354 non-null float64
population         8354 non-null float64
dtypes: float64(3), object(5)
memory usage: 587.4+ KB


In [11]:
# save the dataframe as excel file to be used in tableau

co2_gdp_population_data.to_excel("co2_gdp_population_data.xlsx",index=False)

# check first few rows of the dataframe

co2_gdp_population_data.head(1)

,Country Name,Country Code,Region,IncomeGroup,year,co2_emissions,gdp_current_usd,population
1,Afghanistan,AFG,South Asia,Low income,1960,414.371,5.377778e+08,8996351.0


### Data Exploration

In [12]:
# drop non-numeric columns from the dataframe to calculate spearman correlation between the variables

co2_gdp_population_data_df = co2_gdp_population_data.drop(['Country Name','Country Code','Region','IncomeGroup'], axis=1)
co2_gdp_population_data_df.corr()

,co2_emissions,gdp_current_usd,population
co2_emissions,1.000000,0.764050,0.613163
gdp_current_usd,0.764050,1.000000,0.318355
population,0.613163,0.318355,1.000000


### Data Preparation

In [13]:
## Divide the data between train and test

# convert year to numeric 

co2_gdp_population_data['year'] = co2_gdp_population_data['year'].astype('int')

# drop unnecessary columns from the dataframe

df = co2_gdp_population_data.drop(['Country Name','Country Code','Region','IncomeGroup'], axis=1)

# using year as filter define train data as data before 2011 and assign rest as test data 

train = df[co2_gdp_population_data['year']<=2011]
test = df[co2_gdp_population_data['year']>2011]

# save the predictor variables and remove unnecessary columns

train_x = train.drop(['year','co2_emissions'],axis=1)
test_x = test.drop(['year','co2_emissions'],axis=1)

# save the response variable 

train_y = train['co2_emissions']
test_y = test['co2_emissions']

## Model Building

### Multiple Linear Regression

In [14]:
## Apply multiple linear regression

# Save linear regression function

regr = linear_model.LinearRegression()

# train the model using the training sets

regr.fit(train_x, train_y)

# make predictions using the testing set

y_pred = regr.predict(test_x)

# print the coefficients

print('Coefficients: \n', regr.coef_)

# print explained variance score: 1 is perfect prediction

print('Linear Regression Test Data R-2 score: %.2f' % r2_score(test_y, y_pred))

Coefficients: 
 [  4.49175431e-07   1.79507337e-03]
Linear Regression Test Data R-2 score: 0.78


### Random Forest Regressor

In [15]:
# apply random forest regression on training set (ensemble of decision trees)

rf = RandomForestRegressor(n_estimators=500, oob_score=True, random_state=0)
rf.fit(train_x, train_y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=1,
           oob_score=True, random_state=0, verbose=0, warm_start=False)

### Predictions

In [16]:
## Making Predictions

# make predictions on test dataset 

predicted_test = rf.predict(test_x)

# calculate mean squared error, spearman and pearson correlation on test set

test_score = r2_score(test_y, predicted_test)

# print the results

print(f'Random Forest Test data R-2 score: {test_score:>5.3}')

Random Forest Test data R-2 score: 0.875


### Test Data Export

In [17]:
# Append the predicted values by random forest regressor for co2 emissions as a column in the test data

test['predicted_co2_emmissions'] = predicted_test

# check first few rows of dataframe

test.head(10)

,year,co2_emissions,gdp_current_usd,population,predicted_co2_emmissions
11285,2012,10755.311,2.053654e+10,30696958.0,35583.027860
11286,2012,33399.036,1.153984e+11,25096150.0,164925.554536
11287,2012,4910.113,1.231978e+10,2900401.0,6746.179900
11288,2012,487.711,3.146152e+09,82431.0,534.105884
11289,2012,176386.367,3.748180e+11,8900453.0,56988.443630
11290,2012,192356.152,5.459824e+11,42096739.0,289406.167260
11291,2012,5694.851,1.061932e+10,2881922.0,6295.798960
11293,2012,524.381,1.216046e+09,96777.0,524.417670
11294,2012,388126.281,1.538194e+12,22728254.0,398987.509628
11295,2012,62272.994,4.074516e+11,8429991.0,62131.909842


In [18]:
# Also append the country name column to the test dataset 

co2_gdp_population_data['year'] = co2_gdp_population_data['year'].astype('int')
dd = co2_gdp_population_data[co2_gdp_population_data['year']>2011]
test['Country Name'] = dd['Country Name']

In [19]:
# save the test dataset to be used for plotting in tableau

test.to_excel("test.xlsx",index=False)